In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("data/inspections.csv")

In [4]:
data.head().T

,0,1,2,3,4
pastSerious,0,0,0,0,0
pastCritical,0,0,0,0,0
timeSinceLast,2,2,2,2,2
ageAtInspection,1,1,1,1,1
consumption_on_premises_incidental_activity,0,0,0,0,0
tobacco_retail_over_counter,1,0,0,0,0
temperatureMax,53.4967,59.0467,59.0467,56.1533,52.73
heat_burglary,26.9924,13.9766,12.6112,35.9064,9.53079
heat_sanitation,37.7488,15.4123,8.32033,38.1866,2.12679
heat_garbage,12.7686,12.8955,8.00422,26.2386,3.40157
